<a href="https://colab.research.google.com/github/Haseeb55859/Artificial-intelligence-AI-/blob/main/NOTEBOOK/PHASE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use a clean mount point**

In [9]:
from google.colab import drive

# Use a new empty folder to mount
drive.mount('/content/drive_new')


Mounted at /content/drive_new


**Set Dataset Paths (Skin Cancer 9-class)**

In [11]:
import os

# Base folder
base_dir = "/content/drive_new/MyDrive/datasets/skin_cancer9"

# Check exact contents
print("Contents of base_dir:")
for f in os.listdir(base_dir):
    print(repr(f))  # use repr() to see spaces/special characters


Contents of base_dir:
'train'
'test'
'val'
'manifest.csv'


**ImageDataGenerator**

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- Directories ---
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")
test_dir  = os.path.join(base_dir, "test")

# --- Parameters ---
IMG_HEIGHT = 224
IMG_WIDTH  = 224
BATCH_SIZE = 32
SEED = 42

# -------------------------------
# 1️⃣ Data generators

# Training generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Validation generator (only rescale)
val_datagen = ImageDataGenerator(rescale=1./255)

# Test generator (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)

# -------------------------------
# 2️⃣ Create generators

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # MULTI-CLASS (9 classes)
    shuffle=True,
    seed=SEED
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)


Found 1907 images belonging to 9 classes.
Found 332 images belonging to 9 classes.
Found 118 images belonging to 9 classes.


**Build CNN Model for 9-class Skin Cancer Dataset**

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# -------------------------------
# Build Medium CNN for 9 classes
def build_skin_cnn(input_shape=(224,224,3), num_classes=9):
    model = Sequential([

        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Block 2
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Block 3
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Block 4 (optional deeper)
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Dense layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),

        Dense(num_classes, activation='softmax')  # Multi-class output
    ])
    return model

# -------------------------------
# Create model
model = build_skin_cnn()
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    12,845,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,237,961 (50.50 MB)

 Trainable params: 13,237,001 (50.50 MB)

 Non-trainable params: 960 (3.75 KB)

**Compile Model**

In [14]:
from tensorflow.keras.optimizers import Adam

# Compile model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',   # Multi-class
    metrics=['accuracy']
)

print("Model compiled for 9-class classification.")


Model compiled for 9-class classification.


**CallBacks**

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os

# Folder to save best model
checkpoint_path = "/content/drive_new/MyDrive/skin_cancer_models/best_skin_cnn.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-7)
]

print("Callbacks ready. Model checkpoint path:", checkpoint_path)


Callbacks ready. Model checkpoint path: /content/drive_new/MyDrive/skin_cancer_models/best_skin_cnn.h5


** Training Model**

In [16]:
# Training parameters
EPOCHS = 5

# Fit model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.2792 - loss: 3.4128 

60/60 ━━━━━━━━━━━━━━━━━━━━ 813s 13s/step - accuracy: 0.2801 - loss: 3.3978 - val_accuracy: 0.0602 - val_loss: 8.1731 - learning_rate: 1.0000e-04
Epoch 2/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 47s 764ms/step - accuracy: 0.4268 - loss: 1.6920 - val_accuracy: 0.0602 - val_loss: 13.2861 - learning_rate: 1.0000e-04
Epoch 3/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 46s 772ms/step - accuracy: 0.4310 - loss: 1.6209 - val_accuracy: 0.0602 - val_loss: 17.4513 - learning_rate: 1.0000e-04
Epoch 4/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 45s 748ms/step - accuracy: 0.4087 - loss: 1.6412 - val_accuracy: 0.0602 - val_loss: 18.7621 - learning_rate: 1.0000e-04
Epoch 5/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 46s 772ms/step - accuracy: 0.4642 - loss: 1.5276 - val_accuracy: 0.0602 - val_loss: 16.3076 - learning_rate: 5.0000e-05
